In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
DATA_FINAL_DIR = ""
TEST_FIGURE_DIR = ""
FEATURE_DIR = os.path.join(DATA_FINAL_DIR, "Extracted_Features_Final")
FIG_DIR = os.path.join(TEST_FIGURE_DIR, "Feature_Analysis_Figures")
os.makedirs(FIG_DIR, exist_ok=True)

In [ ]:
df_eda = pd.read_csv(os.path.join(FEATURE_DIR, "features_eda.csv"))
df_hr = pd.read_csv(os.path.join(FEATURE_DIR, "features_hr.csv"))
df_temp = pd.read_csv(os.path.join(FEATURE_DIR, "features_temp.csv"))
df_acc = pd.read_csv(os.path.join(FEATURE_DIR, "features_acc.csv"))

In [ ]:
def map_phase_category(phase):
    if phase == "Baseline":
        return "Baseline"
    elif phase in ["Descriptive_Stress", "Stroop_Stress", "Math_Stress"]:
        return "Stress"
    elif phase == "Intervention":
        return "Intervention"
    elif phase == "Post-Relaxation":
        return "Relaxation"
    else:
        return "???"

In [ ]:
df_eda["phase_category"] = df_eda["phase"].apply(map_phase_category)
df_hr["phase_category"] = df_hr["phase"].apply(map_phase_category)
df_temp["phase_category"] = df_temp["phase"].apply(map_phase_category)
df_acc["phase_category"] = df_acc["phase"].apply(map_phase_category)

In [ ]:
print(df_eda.groupby(["phase", "phase_category"]).size().reset_index(name="count"))
print(df_eda["phase_category"].value_counts())

In [ ]:
all_stats = []

for signal, df, cols in [
    ("EDA", df_eda, eda_feature_cols),
    ("HR", df_hr, hr_feature_cols),
    ("TEMP", df_temp, temp_feature_cols),
    ("ACC", df_acc, acc_feature_cols),
]:

    stats_df = df[cols].describe().T
    stats_df["cv"] = stats_df["std"] / stats_df["mean"] * 100
    stats_df["skew"] = df[cols].skew()
    stats_df["kurtosis"] = df[cols].kurtosis()
    stats_df["signal"] = signal
    print(stats_df[["mean", "std", "min", "max", "cv", "skew"]].round(3).to_string())

    all_stats.append(stats_df)


df_all_stats = pd.concat(all_stats)
df_all_stats.to_csv(os.path.join(FEATURE_DIR, "feature_statistics.csv"))

In [ ]:
from scipy.stats import kruskal, mannwhitneyu

In [ ]:
test_features = {
    "eda_scl_mean": df_eda,
    "eda_scr_rate": df_eda,
    "eda_scr_count":  df_eda,
    "eda_phasic_mean": df_eda,
    "eda_slope": df_eda,
    "hr_mean": df_hr,
    "hr_std": df_hr,
    "hr_rmssd": df_hr,
    "hr_pnn50":   df_hr,
    "temp_mean": df_temp,
    "temp_slope": df_temp,
    "temp_std": df_temp,
    "acc_enmo_mean": df_acc,
    "acc_activity_level":   df_acc,
    "acc_magnitude_mean": df_acc,
}

In [ ]:
kruskal_results = []

for feature, df in test_features.items():
    if feature not in df.columns:
        continue

    groups = [
        df[df["phase_category"] ==  phase][feature].dropna() for phase in phase_order
    ]

    if all(len(g) > 0 for g in groups):
        stat, p_value = kruskal(*groups)
        sig = (
            "---"
            if p_value < 0.001
            else "--" if p_value < 0.01 else "-" if p_value < 0.05 else ""
        )

        kruskal_results.append(
            {
                "feature": feature,
                "H_statistic": stat,
                "p_value": p_value,
                "significant": sig,
            }
        )

        print(f"{feature:25s} H={stat:8.2f}, p={p_value:.2e} {sig}")

df_kruskal = pd.DataFrame(kruskal_results)
df_kruskal.to_csv(os.path.join(FEATURE_DIR, "kruskal_wallis_results.csv"), index=False)



mannwhitney_results = []

for feature, df in test_features.items():
    if feature not in df.columns:
        continue

    baseline = df[df["phase_category"] == "Baseline"][feature].dropna()
    stress = df[df["phase_category"] == "Stress"][feature].dropna()

    if len(baseline) > 0 and len(stress) > 0:
        stat, p_value = mannwhitneyu(baseline, stress, alternative="two-sided")
        sig = (
            "***"
            if p_value < 0.001
            else "**" if p_value < 0.01 else "*" if p_value < 0.05 else ""
        )

        n1, n2 = len(baseline), len(stress)
        effect_size = 1 - (2 * stat) / (n1 * n2)
        direction = "up" if stress.mean() > baseline.mean() else "down"

        mannwhitney_results.append(
            {
                "feature": feature,
                "baseline_mean": baseline.mean(),
                "stress_mean": stress.mean(),
                "direction": direction,
                "U_statistic": stat,
                "p_value": p_value,
                "effect_size": effect_size,
                "significant": sig,
            }
        )

        print(
            f"{feature:25s} Baseline={baseline.mean():8.3f}, Stress={stress.mean():8.3f} {direction}, p={p_value:.2e} {sig}"
        )

df_mannwhitney = pd.DataFrame(mannwhitney_results)
df_mannwhitney.to_csv(
    os.path.join(FEATURE_DIR, "mannwhitney_baseline_vs_stress.csv"), index=False
)

In [ ]:
if len(mannwhitney_results) > 0:
    df_summary = pd.DataFrame(mannwhitney_results)
    df_summary = df_summary.sort_values("p_value")

    print("\nTop features differentiating Baseline vs Stress (sorted by p-value):")
    print(
        df_summary[
            [
                "feature",
                "baseline_mean",
                "stress_mean",
                "direction",
                "p_value",
                "effect_size",
                "significant",
            ]
        ].to_string(index=False)
    )

    sig_features = df_summary[df_summary["significant"] != ""]["feature"].tolist()
    print(f"\n\nSignificant features (p < 0.05): {len(sig_features)}")
    for f in sig_features:
        row = df_summary[df_summary["feature"] == f].iloc[0]
        print(
            f"  {f}: {row['direction']} in stress (effect size = {row['effect_size']:.3f})"
        )

In [ ]:
check_features_map = {
    "eda_scl_mean": df_eda,
    "eda_scr_count": df_eda,
    "eda_phasic_mean": df_eda,
    "hr_mean": df_hr,
    "hr_rmssd": df_hr,
    "hr_pnn50": df_hr,
    "temp_mean": df_temp,
    "temp_slope": df_temp,
    "acc_enmo_mean": df_acc,
    "acc_magnitude_mean": df_acc,
    "acc_activity_level": df_acc,
    "acc_sma": df_acc,
}

fig, axes = plt.subplots(3, 4, figsize=(16, 12))
axes = axes.flatten()

for ax, (feature, df) in zip(axes, check_features_map.items()):
    if feature not in df.columns:
        ax.set_visible(False)
        continue

    data = df[feature].dropna()

    ax.hist(data, bins=50, edgecolor="black", alpha=0.7)
    ax.axvline(
        data.mean(), color="red", linestyle="--", label=f"Mean: {data.mean():.2f}"
    )
    ax.axvline(
        data.median(),
        color="green",
        linestyle="--",
        label=f"Median: {data.median():.2f}",
    )

    ax.set_title(f"{feature}\n(skew={data.skew():.2f})", fontsize=10)
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)

plt.suptitle(
    "feature distribution", fontsize=14, fontweight="bold"
)
plt.tight_layout()
plt.savefig(
    os.path.join(FIG_DIR, "feature_distributions_histograms.png"),
    dpi=150,
    bbox_inches="tight",
)
plt.show()